# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121105e+02     1.732582e+00
 * time: 0.8423008918762207
     1     1.090833e+01     9.499619e-01
 * time: 3.48396897315979
     2    -1.188398e+01     1.115551e+00
 * time: 4.208762884140015
     3    -3.421156e+01     8.958310e-01
 * time: 5.276172876358032
     4    -4.775311e+01     6.695822e-01
 * time: 6.722339868545532
     5    -5.703762e+01     2.041665e-01
 * time: 7.753896951675415
     6    -5.986037e+01     1.373281e-01
 * time: 8.443837881088257
     7    -6.096018e+01     5.357052e-02
 * time: 9.139747858047485
     8    -6.139264e+01     6.679423e-02
 * time: 9.828896045684814
     9    -6.166917e+01     5.022997e-02
 * time: 10.54561185836792
    10    -6.187675e+01     3.090067e-02
 * time: 11.237627029418945
    11    -6.202762e+01     1.900593e-02
 * time: 11.931596994400024
    12    -6.209194e+01     1.594464e-02
 * time: 12.630012035369873
    13    -6.215076e+01     1.293505e-02
 * time: 13.330188989639282
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671075
    AtomicLocal         -18.8557697
    AtomicNonlocal      14.8522652
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336823

    total               -62.261666460925
